# Load Data

In [1]:
from proj1_helpers import *
from implementations import *
from cross_validation import *
import numpy as np


%load_ext autoreload
%autoreload 2

oy, otx, oids = load_csv_data('Data/train.csv', False)
otest_y, otest_tx, otest_ids= load_csv_data('Data/test.csv', False)

In [10]:
from preprocessing import *

for nbColumns in range(10,11):
    y, tx, ids = oy, otx, oids
    test_y, test_tx,test_ids = otest_y, otest_tx, otest_ids




    # Preprocessing

    #print(test_y.shape)



    PRI_jet_num_colomn_train = tx[:, 22]
    PRI_jet_num_colomn_test = test_tx[:, 22]


    y_clustered = list()
    tx_clustered = list()
    ids_clustered = list()
    test_y_clustered = list()
    test_tx_clustered = list()
    test_ids_clustered = list()

    for i in range(4):
        indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_train) if a == i]

        y_clustered.append(y[indices])
        tx_clustered.append(addColumns(tx[indices], nbColumns))
        ids_clustered.append(ids[indices])

        test_indices = [ind for ind,a in enumerate(PRI_jet_num_colomn_test) if a == i]
        test_y_clustered.append(test_y[test_indices])
        
        test_tx_clustered.append(addColumns(test_tx[test_indices],nbColumns))
        print("Len tx : ",len(test_tx_clustered[i][0]))
        test_ids_clustered.append(test_ids[test_indices])

        #delete colinear columns
        indices_to_delete = list()
        for col in range(tx_clustered[i].shape[1]):
            if min(tx_clustered[i][:, col]) == max(tx_clustered[i][:, col]):
                indices_to_delete.append(col)

        tx_clustered[i] = np.delete(tx_clustered[i], indices_to_delete, 1)
        test_tx_clustered[i] = np.delete(test_tx_clustered[i], indices_to_delete, 1)



    # Cross-Validation & Weight computation

    from cross_validation import *
    w = list()
    result = list()

    loss = 0
    perGood = 0
    for i in range(4):
        wi, loss_tr, loss_te, perGoodI = cross_validation_run(tx_clustered[i], y_clustered[i])
        w.append(wi)
        loss += loss_te/4
        perGood += perGoodI/4
        test_y_clustered[i] = predict_labels(w[i], test_tx_clustered[i])
        
    print("Number of columns : ", nbColumns, " loss : ", loss, "Percentage of true Y : ", perGood)
   

Len tx :  270
Len tx :  270
Len tx :  270
Len tx :  270
Number of columns :  10  loss :  4.57594964754e+26 Percentage of true Y :  0.822293129522


# Post Processing

In [11]:
test_ids = [item for sublist in test_ids_clustered for item in sublist]
y_pred = [item for sublist in test_y_clustered for item in sublist]

test_ids, y_pred = zip(*sorted(zip(test_ids, y_pred)))

# Output Data

In [12]:
name = 'third-submission'
create_csv_submission(test_ids, y_pred, name)